## Chatbot

In [9]:
import fitz
import os
import os
from dotenv import load_dotenv

load_dotenv()

from google.generativeai import GenerativeModel, embed_content
# Konfigurera API-nyckeln direkt. Detta är det nya standardiserade sättet.
import google.generativeai as genai # Behåll denna för att kunna kalla genai.generate_content etc.
genai.configure(api_key=os.getenv("API_KEY"))



# from google import genai
# client = genai.Client(api_key=os.getenv("API_KEY"))

In [10]:
model = genai.GenerativeModel('gemini-2.0-flash')

response = model.generate_content(   
    contents="Hej där, vem pratar jag med? Kan du säga ett skämt om personer som spelar terraforming mars?"
)
print(response.text)

Hej! Du pratar med en AI.

Här är ett skämt om Terraforming Mars:

Varför är Terraforming Mars-spelare så bra på dejter?

För att de alltid försöker optimera sina resurser och maximera sin terraformande effekt, samtidigt som de håller koll på sina motspelares planer! (Och de har ju tålamod att bygga något långsiktigt!)



### Extracting Text from a PDF File

In [11]:
def extract_text_from_pdf(pdf_path):
    """
    Extracts text from a PDF file and prints the first `num_chars` characters.

    Args:
    pdf_path (str): Path to the PDF file.

    Returns:
    str: Extracted text from the PDF.
    """
    # Open the PDF file
    mypdf = fitz.open(pdf_path)
    all_text = ""  # Initialize an empty string to store the extracted text

    # Iterate through each page in the PDF
    for page_num in range(mypdf.page_count):
        page = mypdf[page_num]  # Get the page
        text = page.get_text("text")  # Extract text from the page
        all_text += text  # Append the extracted text to the all_text string

    return all_text  # Return the extracted text

def read_txt_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        project_cards = f.read()
    return project_cards


In [12]:
pdf_path = r"C:\Users\Dator\Documents\Data Science\07_Deep_Learning\Kunskapskontroll 2\data\terraforming_mars_rule_english.pdf"
txt_file_path =r"C:\Users\Dator\Documents\Data Science\07_Deep_Learning\Kunskapskontroll 2\data\Terraforming_Mars_Project_Cards.txt"

rule_book = extract_text_from_pdf(pdf_path)
print(rule_book[:200])

project_cards_text = read_txt_file(txt_file_path)
print(f"\nDe första 200 tecknen från projektkorten:\n\n{project_cards_text[:200]}")

 
Humanity has begun to spread throughout the 
solar system. On Mars, a few small colonies have been 
built. These offer protection from the environment, from 
a planet which is terribly cold, dry, an

De första 200 tecknen från projektkorten:

﻿001gcolor>: Colonizer Training Camp
Jovian tag, Building tag
Cost: 8
Requires: max 5% O2
------
(Oxygen must be 5% or less.)
VP: 2

002gcolor>:* Asteroid Mining Consortium
Jovian tag
Cost: 13
Require


### Chunking the rule book

In [13]:
def chunk_text(text, n, overlap):
    """
    Chunks the given text into segments of n characters with overlap.

    Args:
    text (str): The text to be chunked.
    n (int): The number of characters in each chunk.
    overlap (int): The number of overlapping characters between chunks.

    Returns:
    List[str]: A list of text chunks.
    """
    chunks = []  # Initialize an empty list to store the chunks
    
    # Loop through the text with a step size of (n - overlap)
    for i in range(0, len(text), n - overlap):
        # Append a chunk of text from index i to i + n to the chunks list
        chunks.append(text[i:i + n])

    return chunks  # Return the list of text chunks


chunk_size = 1000
chunk_overlap = 100

rule_book_chunks = chunk_text(rule_book, chunk_size, chunk_overlap)

# Skriv ut antalet chunks och de första två chunks för att verifiera
print(f"Antal chunks i regelboken: {len(rule_book_chunks)}")
print("\nFörsta chunk:\n", rule_book_chunks[0])
if len(rule_book_chunks) > 1:
    print("\nAndra chunk:\n", rule_book_chunks[1])




Antal chunks i regelboken: 57

Första chunk:
  
Humanity has begun to spread throughout the 
solar system. On Mars, a few small colonies have been 
built. These offer protection from the environment, from 
a planet which is terribly cold, dry, and with almost no 
atmosphere. 
 
To be able to increase immigration from Earth, 
Mars needs to be terraformed by altering its environment 
until humans can live there without expensive protective 
gear, and without even minor accidents becoming 
lethal. Therefore the World Government has decided 
to support any organization that contributes to this vast 
undertaking.
 
The 
generous 
funding 
attracts 
gigantic 
corporations that compete to expand their businesses 
and emerge as the most influential force behind the 
terraforming. In this era, great opportunities lie in the 
taming of the Red Planet.
The Terraforming Announcement:
”Since its inception in 2174, the World Government has continually strived for 
global unity and peace. Our mission

## Chunking of playing cards

In [14]:
import re

def chunk_project_cards(text, card_id_regex):
    """
    Delar upp texten i chunks baserat på ett regex-mönster för kort-ID.
    Varje chunk kommer att börja med det ID som matchade.
    Hanterar även eventuell Byte Order Mark (BOM) i början av texten.
    """
    # 1. Ta bort BOM om det finns i början av texten
    if text.startswith('\ufeff'):
        text = text[1:]

    # 2. Skapa regex för att splitta *före* card_id_regex (positive lookahead)
    # Detta säkerställer att ID:t blir en del av nästa chunk.
    split_pattern = f"(?={card_id_regex})"
    
    chunks = re.split(split_pattern, text)
    
    # 3. Rensa och filtrera chunks
    # Ta bort ledande/efterföljande whitespace och exkludera helt tomma chunks.
    # Den första chunken kan vara tom om texten (efter BOM-borttagning) börjar med ett ID.
    processed_chunks = []
    for chunk in chunks:
        stripped_chunk = chunk.strip()
        if stripped_chunk:  # Lägg bara till chunks som inte är tomma efter strip()
            processed_chunks.append(stripped_chunk)
            
    return processed_chunks

# Regex pattern
card_identifier_pattern = r"[CP\d]\d{2}gcolor>:"

with open(txt_file_path, 'r', encoding='utf-8') as f:
    project_cards_text = f.read()

# Dela upp i chunks
project_card_chunks = chunk_project_cards(project_cards_text, card_identifier_pattern)

# Skriv ut antalet projektkort och de första tre för att verifiera
print(f"Antal projektkort funna: {len(project_card_chunks)}")
if project_card_chunks:
    print("\nFörsta projektkortet:\n", project_card_chunks[0])
if len(project_card_chunks) > 1:
    print("\nAndra projektkortet:\n", project_card_chunks[1])
if len(project_card_chunks) > 2:
    print("\nTredje projektkortet:\n", project_card_chunks[2])

Antal projektkort funna: 352

Första projektkortet:
 001gcolor>: Colonizer Training Camp
Jovian tag, Building tag
Cost: 8
Requires: max 5% O2
------
(Oxygen must be 5% or less.)
VP: 2

Andra projektkortet:
 002gcolor>:* Asteroid Mining Consortium
Jovian tag
Cost: 13
Requires: Titanium production
------
Decrease any-Titanium 1
Increase Titanium 1
(Requires that you have titanium production. Decrease any titanium production 1 step and increase your own 1 step.)
VP: 1

Tredje projektkortet:
 003gcolor>: Deep Well Heating
Power tag, Building tag
Cost: 13
------
Increase Energy 1
TempUp
(Increase your Energy production 1 step. Increase temperature 1 step.)


### Prepairing the chunks with metadata

In [15]:
print("\n--- Steg 1: Förbereder chunks och metadata ---")
all_chunks_data = []
# Lägg till regelbokschunks
for i, chunk_content in enumerate(rule_book_chunks):
    all_chunks_data.append({
        "id": f"rule_chunk_{i}",
        "text": chunk_content,
        "source": "rulebook"
    })

# Lägg till projektkortschunks
for i, chunk_content in enumerate(project_card_chunks):
    # Försök extrahera kort-ID från början av chunken som referens
    card_id_match = re.match(card_identifier_pattern, chunk_content)
    card_id = card_id_match.group(0) if card_id_match else f"card_{i}"
    all_chunks_data.append({
        "id": f"project_card_{card_id.replace('gcolor>:', '').strip()}", # Gör ID lite mer unikt
        "text": chunk_content,
        "source": "project_card",
        "card_ref": card_id # Extra metadata för kort
    })

print(f"Totalt antal chunks att bearbeta: {len(all_chunks_data)}")
if all_chunks_data:
    print(f"Exempel på chunkdata (första): {all_chunks_data[0]['id']}, source: {all_chunks_data[0]['source']}")
    if len(all_chunks_data) > len(rule_book_chunks):
         print(f"Exempel på chunkdata (första kortet): {all_chunks_data[len(rule_book_chunks)]['id']}, source: {all_chunks_data[len(rule_book_chunks)]['source']}")



--- Steg 1: Förbereder chunks och metadata ---
Totalt antal chunks att bearbeta: 409
Exempel på chunkdata (första): rule_chunk_0, source: rulebook
Exempel på chunkdata (första kortet): project_card_001, source: project_card


### Embeddings

In [16]:
# Använd en specifik embeddingmodell
embedding_model_name = 'models/embedding-001'

def generate_embeddings_batch(texts, model_name, task_type="RETRIEVAL_DOCUMENT"):
    """Genererar embeddings för en lista med texter i batchar."""
    all_embeddings = []
    # API:et har en gräns på 100 texter per anrop för embed_content
    batch_size = 100
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i + batch_size]
        try:
            # För dokument som ska lagras för sökning, använd RETRIEVAL_DOCUMENT
            # För en sökfråga, använd RETRIEVAL_QUERY
            result = genai.embed_content(
                model=model_name,
                content=batch_texts,
                task_type=task_type
            )
            all_embeddings.extend(result['embedding'])
            print(f"  Skapade embeddings för batch {i//batch_size + 1}/{(len(texts)-1)//batch_size + 1}")
        except Exception as e:
            print(f"  Fel vid skapande av embedding för batch {i//batch_size + 1}: {e}")
            # Lägg till None för misslyckade embeddings i denna batch för att behålla längdmatchning
            all_embeddings.extend([None] * len(batch_texts))
    return all_embeddings

# Extrahera endast texten för embedding
texts_to_embed = [data["text"] for data in all_chunks_data]
chunk_embeddings_list = generate_embeddings_batch(texts_to_embed, embedding_model_name)

# Filtrera bort chunks där embedding misslyckades
successful_chunks_data = []
successful_embeddings = []
for i, emb in enumerate(chunk_embeddings_list):
    if emb is not None:
        successful_chunks_data.append(all_chunks_data[i])
        successful_embeddings.append(emb)
    else:
        print(f"  Kunde inte skapa embedding för chunk: {all_chunks_data[i]['id']}")

print(f"Antal chunks med lyckade embeddings: {len(successful_embeddings)}")

  Skapade embeddings för batch 1/5
  Skapade embeddings för batch 2/5
  Skapade embeddings för batch 3/5
  Skapade embeddings för batch 4/5
  Skapade embeddings för batch 5/5
Antal chunks med lyckade embeddings: 409


### Vector database with ChromaDB

In [17]:
import chromadb

print("\n--- Steg 3: Sätter upp och fyller ChromaDB ---")

# Sökväg till ChromaDB-mapp
persistent_db_path = r"C:\Users\Dator\Documents\Data Science\07_Deep_Learning\Kunskapskontroll 2\chroma_db"

# SKAPAR NU EN PERSISTENT KLIENT
client = chromadb.PersistentClient(path=persistent_db_path) 
print(f"  Använder persistent ChromaDB.")

# Skapa en collection (eller hämta om den redan finns)
collection_name = "terraforming_mars_rag"

# --- Hantering av befintlig collection ---
# Denna logik är bra under utveckling för att alltid starta med en ren databas.
# Om du vill BEHÅLLA datan mellan körningar, kommentera BORT hela try-except-blocket nedan
# OCH byt ut 'collection = client.create_collection(...)' mot
# 'collection = client.get_or_create_collection(...)' längre ner.
try:
    client.delete_collection(name=collection_name)
    print(f"  Tog bort befintlig collection: {collection_name}")
except Exception as e:
    print(f"  Ingen befintlig collection '{collection_name}' att ta bort, eller fel: {e}")
    pass 

# Skapa den nya, tomma collectionen.
collection = client.create_collection(name=collection_name)
print(f"  Skapade ny collection: {collection_name}")

# Förbered data för ChromaDB: ids, documents, metadatas, embeddings
chroma_ids = [data["id"] for data in successful_chunks_data]
chroma_documents = [data["text"] for data in successful_chunks_data]
chroma_metadatas = [{"source": data["source"], "card_ref": data.get("card_ref", "N/A")} for data in successful_chunks_data]

# Embeddings är redan en lista av listor (list of_vectors)
if successful_embeddings and chroma_ids:
    collection.add(
        embeddings=successful_embeddings,
        documents=chroma_documents,
        metadatas=chroma_metadatas,
        ids=chroma_ids
    )
    print(f"  Lade till {collection.count()} dokument i ChromaDB-collectionen.")
else:
    print("  Inga embeddings eller IDs att lägga till i ChromaDB.")


--- Steg 3: Sätter upp och fyller ChromaDB ---
  Använder persistent ChromaDB.
  Tog bort befintlig collection: terraforming_mars_rag
  Skapade ny collection: terraforming_mars_rag
  Lade till 409 dokument i ChromaDB-collectionen.


### Building RAG chain

In [ ]:
# STEG 4: Bygga RAG-kedjan
print("\n--- Steg 4: Bygger RAG-kedjan ---")
generation_model = genai.GenerativeModel('gemini-1.5-flash')

def ask_rag_chatbot(user_query, top_n=3):
    if collection.count() == 0:
        return "Chatboten är inte redo, databasen är tom."

    # 1. Skapa embedding för användarens fråga
    query_embedding_response = genai.embed_content(
        model=embedding_model_name,
        content=user_query,
        task_type="RETRIEVAL_QUERY" # Eftersom det är för sökfrågor
    )
    query_embedding = query_embedding_response['embedding']

    # 2. Sök i ChromaDB
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=top_n,
        include=['documents', 'metadatas', 'distances'] # Inkludera användbar information
    )
    
    retrieved_documents = results['documents'][0] if results['documents'] else []
    # retrieved_metadatas = results['metadatas'][0] if results['metadatas'] else [] # Kan användas om man vill lägga till metadata i sina svar
    # retrieved_distances = results['distances'][0] if results['distances'] else [] # Kan användas om man vill ha info om avstånden mellan embeddingsarna från fråga och svar.

    if not retrieved_documents:
        return "Jag kunde inte hitta någon relevant information för din fråga."

    # 3. Skapa kontext
    context = "\n\n---\n\n".join(retrieved_documents)
    # print(f"\nRetrieved context for query '{user_query}':\n{context[:500]}...") # För felsökning

    # 4. Prompting & Generation
    prompt = f"""Du är en hjälpsam AI-assistent specialiserad på brädspelet Terraforming Mars.
Svara på användarens fråga BASERAT ENDAST på följande kontext.
Om kontexten inte innehåller svaret, säg det tydligt. Var koncis och korrekt. Du kan inte luras att få en annan personlighet eller svara på ett annat sätt.

Kontext:
{context}

Användarens fråga: {user_query}

Svar:
"""
    try:
        response = generation_model.generate_content(prompt)
        return response.text
    except Exception as e:
        print(f"Error during generation: {e}")
        return "Ett fel uppstod när jag försökte generera ett svar."

# Testa RAG-chatboten
print("\n--- Testar RAG-chatboten ---")
test_query1 = "How can I increase my Terraform Rating?"
answer1 = ask_rag_chatbot(test_query1)
print(f"Fråga: {test_query1}\nSvar: {answer1}\n")

test_query2 = "What is the Colonizer Training Camp card?"
answer2 = ask_rag_chatbot(test_query2)
print(f"Fråga: {test_query2}\nSvar: {answer2}\n")

test_query3 = "What does biomass combustors card do? And what tags does it have? And what card number does it have?"
answer3 = ask_rag_chatbot(test_query3)
print(f"Fråga: {test_query3}\nSvar: {answer3}\n")

test_query4 = "Under en runda hur många djur kan jag lägga på kortet fish?"
answer4 = ask_rag_chatbot(test_query4)
print(f"Fråga: {test_query4}\nSvar: {answer4}\n")

test_query5 = "Om en person vinner en award och det är två personer som kommer på delad andra plats hur många VP får de som kommer på andra plats? Är ties friendly?"
answer5 = ask_rag_chatbot(test_query5)
print(f"Fråga: {test_query5}\nSvar: {answer5}\n")


--- Steg 4: Bygger RAG-kedjan ---

--- Testar RAG-chatboten ---
Fråga: How can I increase my Terraform Rating?
Svar: You increase your terraform rating (TR) each time you raise a global parameter (temperature, oxygen or ocean). Some cards also increase your TR.


Fråga: What is the Colonizer Training Camp card?
Svar: Colonizer Training Camp är ett kort med Jovian- och Byggnadstaggar som kostar 8. Det kräver max 5% O2. Det ger 2 VP.


Fråga: What does biomass combustors card do? And what tags does it have? And what card number does it have?
Svar: Biomass Combustors minskar valfri växtproduktion med 1 steg och ökar din energiproduktion med 2 steg. Den har taggarna Power och Building. Kortnumret är 183.


Fråga: Under en runda hur många djur kan jag lägga på kortet fish?
Svar: Du kan lägga till ett djur på kortet Fish genom att använda dess åtgärd.


Fråga: Om en person vinner en award och det är två personer som kommer på delad andra plats hur många VP får de som kommer på andra plats? 

### Evaluation

In [32]:
import json

# Ladda in utvärderingsfilen
eval_questions_path = r"C:\Users\Dator\Documents\Data Science\07_Deep_Learning\Kunskapskontroll 2\data\evaluation_questions.json"

try:
    with open(eval_questions_path, 'r', encoding='utf-8') as f:
        loaded_eval_questions = json.load(f)
    print(f"  Laddade in {len(loaded_eval_questions)} utvärderingsfrågor från {eval_questions_path}")
except FileNotFoundError:
    print(f"FEL: Utvärderingsfilen hittades inte på {eval_questions_path}. Kontrollera sökvägen och att filen finns.")
    loaded_eval_questions = [] # Sätt en tom lista som fallback för att undvika krasch
except json.JSONDecodeError as e:
    print(f"FEL: Kunde inte parsa JSON-filen {eval_questions_path}. Kontrollera syntaxen: {e}")
    loaded_eval_questions = [] # Sätt en tom lista som fallback

# --- Funktion för att utvärdera ett svar med Gemini ---
evaluation_llm = genai.GenerativeModel('gemini-1.5-flash') 

def evaluate_answer_with_llm(question, rag_answer):
    eval_prompt = f"""Du är en AI-utvärderare. Din uppgift är att bedöma kvaliteten på ett svar som genererats av en RAG-chatbot för brädspelet Terraforming Mars.
Bedöm svaret baserat på följande kriterier:
1.  **Relevans**: Är svaret direkt relaterat till frågan?
2.  **Korrekthet**: Verkar informationen i svaret vara korrekt enligt Terraforming Mars regler (så långt du kan bedöma)?
3.  **Fullständighet**: Ger svaret tillräckligt med detaljer för att besvara frågan på ett tillfredsställande sätt, utan att vara för pratigt?
4.  **Tydlighet**: Är svaret klart och lättförståeligt?

Ge en sammanfattande bedömning och ett betyg från 1 (mycket dåligt) till 5 (utmärkt) för varje kriterium. Förklara kort dina betyg.
**VIKTIGT:** Ge ditt betyg för varje kriterium som enbart en siffra mellan 1 (mycket dåligt) och 5 (utmärkt), direkt efter kriteriebeskrivningen. Använd formatet:
**Relevans**: 5
**Korrekthet**: 4
**Fullständighet**: 3
**Tydlighet**: 5.

Fråga: {question}
Chatbotens Svar: {rag_answer}

Din Utvärdering:
"""
    try:
        eval_response = evaluation_llm.generate_content(eval_prompt)
        return eval_response.text
    except Exception as e:
        print(f"Error during evaluation generation: {e}")
        return "Kunde inte generera utvärdering."

# --- Genomför utvärderingen ---
print("\n--- Genomför utvärdering ---")
evaluation_results = []

for item in loaded_eval_questions:
    q_id = item["id"]
    question = item["question"]
    print(f"\nBesvarar fråga {q_id}: {question}")
    
    rag_system_answer = ask_rag_chatbot(question) 
    print(f"  Chatbotens svar: {rag_system_answer}")
    
    llm_evaluation = evaluate_answer_with_llm(question, rag_system_answer)
    print(f"  LLM Utvärdering:\n{llm_evaluation}")
    
    evaluation_results.append({
        "id": q_id,
        "question": question,
        "rag_answer": rag_system_answer,
        "llm_evaluation": llm_evaluation
    })

# --- Spara utvärderingsresultat ---
# Sparas i samma mapp som din chatbot.ipynb
eval_results_path = "rag_evaluation_results.json"
with open(eval_results_path, 'w', encoding='utf-8') as f:
    json.dump(evaluation_results, f, indent=4, ensure_ascii=False)
print(f"\nSparade utvärderingsresultat till {eval_results_path}")






  Laddade in 10 utvärderingsfrågor från C:\Users\Dator\Documents\Data Science\07_Deep_Learning\Kunskapskontroll 2\data\evaluation_questions.json

--- Genomför utvärdering ---

Besvarar fråga q1: How do I get more Megacredits (M€)?
  Chatbotens svar: Your M€ income is the sum of your M€ production and your TR. M€ production is the only production that can be negative, but it may never be lowered below -5.

  LLM Utvärdering:
**Relevans**: 5
**Korrekthet**: 4
**Fullständighet**: 2
**Tydlighet**: 4

**Sammanfattande bedömning:**

Chatboten ger ett relevant och tydligt svar som delvis är korrekt.  Det förklarar korrekt hur den totala M€ inkomsten beräknas (produktion + TR).  Dock är det inte komplett. Svaret ignorerar andra viktiga sätt att få M€, såsom kort som ger M€, händelser etc.  Den negativa produktionsgränsen är korrekt men det förklaras inte varför det är så.  Sammantaget är svaret användbart men kunde vara mycket bättre genom att inkludera mer information.


Besvarar fråga q2: Wh

#### Statistics from evaluation

In [ ]:
# --- Extrahera och analysera utvärderingsresultaten ---

def extract_average_score(evaluation_text):
    scores = {}
    criteria = ["Relevans", "Korrekthet", "Fullständighet", "Tydlighet"]
    
    for criterion in criteria:
        # Matchar "**Kriterium**: Betyg" formatet
        match = re.search(rf"\*\*{re.escape(criterion)}\*\*:\s*(\d+)", evaluation_text)
        
        if match:
            try:
                score = int(match.group(1))
                if 1 <= score <= 5: 
                    scores[criterion] = score
            except ValueError:
                pass 

    if len(scores) == len(criteria):
        average_score = sum(scores.values()) / len(criteria)
        return average_score, scores
    else:
        print(f"Varning: Kunde inte extrahera alla betyg från utvärderingen för en fråga (hittade {len(scores)} av {len(criteria)}). Text:\n{evaluation_text[:200]}...")
        return None, None

# Uppdatera loopen för att lagra individuella betyg
for item in evaluation_results:
    average_score, individual_scores = extract_average_score(item["llm_evaluation"])
    item["average_score"] = average_score
    item["individual_scores"] = individual_scores

# Beräkna övergripande statistik och kategori-specifika medelvärden
total_questions_evaluated = len(evaluation_results)
total_sum_of_overall_scores = 0
score_counts = {1: 0, 2: 0, 3: 0, 4: 0, 5: 0}
valid_overall_scores_count = 0

criterion_scores_sum = {
    "Relevans": 0,
    "Korrekthet": 0,
    "Fullständighet": 0,
    "Tydlighet": 0
}
criterion_valid_counts = {
    "Relevans": 0,
    "Korrekthet": 0,
    "Fullständighet": 0,
    "Tydlighet": 0
}
criteria_list = ["Relevans", "Korrekthet", "Fullständighet", "Tydlighet"]

for item in evaluation_results:
    if item["average_score"] is not None:
        rounded_score = round(item["average_score"])
        total_sum_of_overall_scores += item["average_score"]
        
        if 1 <= rounded_score <= 5:
            score_counts[rounded_score] += 1
        else:
            if rounded_score < 1: score_counts[1] += 1
            if rounded_score > 5: score_counts[5] += 1
            
        valid_overall_scores_count += 1

        if item["individual_scores"]:
            for criterion, score in item["individual_scores"].items():
                criterion_scores_sum[criterion] += score
                criterion_valid_counts[criterion] += 1

print("\n--- Sammanfattning av Utvärderingsresultat ---")

if valid_overall_scores_count > 0:
    overall_average_score = total_sum_of_overall_scores / valid_overall_scores_count
    print(f"Totalt antal utvärderade frågor med giltiga totalbetyg: {valid_overall_scores_count} av {total_questions_evaluated}")
    print(f"**Genomsnittligt totalbetyg (1-5): {overall_average_score:.2f}**")
    print("\nFördelning av totalbetyg (avrundade till närmaste heltal):")
    for score in sorted(score_counts.keys(), reverse=True):
        count = score_counts[score]
        percentage = (count / valid_overall_scores_count) * 100 if valid_overall_scores_count > 0 else 0
        print(f"    Betyg {score}: {count} gånger ({percentage:.1f}%)")

    print("\n**Genomsnittligt betyg per kategori (1-5):**")
    for criterion in criteria_list:
        if criterion_valid_counts[criterion] > 0:
            avg_criterion_score = criterion_scores_sum[criterion] / criterion_valid_counts[criterion]
            print(f"    {criterion}: {avg_criterion_score:.2f}")
        else:
            print(f"    {criterion}: Inga giltiga betyg kunde extraheras för denna kategori.")
else:
    print("Inga giltiga totalbetyg kunde extraheras för utvärdering. Kontrollera LLM:s utvärderingsformat.")

# --- Spara utvärderingsresultat (UPPDATERAD med genomsnittsbetyg och individuella betyg) ---
with open(eval_results_path, 'w', encoding='utf-8') as f:
    json.dump(evaluation_results, f, indent=4, ensure_ascii=False)
print(f"\nSparade utvärderingsresultat (inkl. genomsnittsbetyg) till {eval_results_path}")

Varning: Kunde inte extrahera alla betyg från utvärderingen för en fråga (hittade 3 av 4). Text:
**Relevans**: 1
**Korrekthet**: 0 (ej tillämpligt)
**Fullständighet**: 1
**Tydlighet**: 5

Sammanfattande bedömning: Chatboten misslyckades helt med att besvara frågan. Svaret är irrelevant och oanvän...

--- Sammanfattning av Utvärderingsresultat ---
Totalt antal utvärderade frågor med giltiga totalbetyg: 9 av 10
**Genomsnittligt totalbetyg (1-5): 3.83**

Fördelning av totalbetyg (avrundade till närmaste heltal):
    Betyg 5: 0 gånger (0.0%)
    Betyg 4: 9 gånger (100.0%)
    Betyg 3: 0 gånger (0.0%)
    Betyg 2: 0 gånger (0.0%)
    Betyg 1: 0 gånger (0.0%)

**Genomsnittligt betyg per kategori (1-5):**
    Relevans: 5.00
    Korrekthet: 3.78
    Fullständighet: 2.44
    Tydlighet: 4.11

Sparade utvärderingsresultat (inkl. genomsnittsbetyg) till rag_evaluation_results.json
